# **Imports Bibliotecas**

---



In [ ]:
import requests
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, silhouette_score, roc_auc_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import graphviz
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier, plot_importance

# **Comunicação API**

---



Credenciais para acesso API

Aqui está um pequeno tutorial sobre como gerar o client_id e client_secret no Spotify:

Passo 1: Criar um aplicativo Spotify para desenvolvedores

Acesse o https://developer.spotify.com/.
Faça login ou crie uma conta, se ainda não tiver uma.
Clique em "Criar um aplicativo" e preencha os detalhes do aplicativo.
Você pode preencher com quaisquer dados, é apenas para cadastro.
Anote o Client ID.
Passo 2: Obter o Client Secret

Na página do seu aplicativo, clique em "Mostrar Client Secret".
Anote o Client Secret.
Importante: Mantenha seu Client Secret em segurança e nunca o compartilhe publicamente.

In [ ]:
CLIENT_ID = '[Adicionar seu client ID]'
CLIENT_SECRET = '[Adicionar seu cliente secret]'

Montagem do request para API

In [ ]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

auth_response_data = auth_response.json()
access_token = auth_response_data['access_token']

headers = {
    'Authorization': f'Bearer {access_token}'
}

# **Funções de Request API**

---

Função para coletar IDs das músicas da playlist

In [ ]:
def get_playlist_tracks(playlist_id):
    track_ids = []
    playlist_url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'

    params = {
        'limit': 100
    }

    while True:
        response = requests.get(playlist_url, headers=headers, params=params)
        if response.status_code != 200:
            print(f"Erro ao buscar faixas da playlist: {response.status_code}")
            break

        playlist_data = response.json()
        tracks = playlist_data.get('items', [])

        for item in tracks:
            track = item['track']
            if track:
                track_ids.append(track['id'])

        next_url = playlist_data.get('next')
        if next_url:
            playlist_url = next_url
        else:
            break

    return track_ids



---



Função para coletar features de cada uma das músicas




In [ ]:
def get_audio_features_for_tracks(track_ids):
    audio_features = []
    features_url = 'https://api.spotify.com/v1/audio-features'

    for i in range(0, len(track_ids), 100):
        track_batch = track_ids[i:i+100]
        params = {
            'ids': ','.join(track_batch)
        }
        response = requests.get(features_url, headers=headers, params=params)

        if response.status_code == 200:
            features_data = response.json().get('audio_features', [])
            audio_features.extend(features_data)
        else:
            print(f"Erro ao buscar audio features: {response.status_code}")

    return audio_features

def get_playlist_audio_features(track_ids):
    if not track_ids:
        print("Nenhuma faixa encontrada na playlist.")
        return []

    audio_features = get_audio_features_for_tracks(track_ids)

    if not audio_features:
        print("Nenhum audio feature encontrado.")
        return []

    return audio_features


def get_audio_features_for_track(track_id):
    features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
    response = requests.get(features_url, headers=headers)

    if response.status_code == 200:
        features_data = response.json()
        return features_data
    else:
        print(f"Erro ao buscar audio features: {response.status_code}")
        return None

# **Coleta dos dados**

---
Id da Playlist


Aqui está uma explicação de como coletar o ID de uma playlist do Spotify:

Passo 1: Abra o Spotify e encontre a playlist desejada.

Passo 2: Clique nos três pontos (...) ao lado do nome da playlist.

Passo 3: Selecione "Compartilhar".

Passo 4: Escolha "Copiar link da playlist".

Passo 5: Cole o link em um editor de texto.

Passo 6: O ID da playlist é a sequência de caracteres após /playlist/ e antes de ?.

Exemplo:


https://open.spotify.com/playlist/71xG6fIgIX0TeqO9Dat7YJ?si=fb05bc723d08473a

Nesse exemplo o id é: fb05bc723d08473a

In [ ]:
playlist_id = '[Insira somente o id da playlist]'



---

Execução das funções de chamada API


1.   Coleta de IDs das músicas
2.   Coleta de features de cada uma das músicas



In [ ]:
track_ids =  get_playlist_tracks(playlist_id)
audio_features = get_playlist_audio_features(track_ids)



---

Função para criar Dataframe com features selecionadas

In [ ]:
def create_df(audio_features):
    df = pd.DataFrame(audio_features)
    df = df[['danceability', 'energy', 'speechiness', 'acousticness',
            'instrumentalness', 'liveness', 'valence', 'tempo','time_signature','mode','loudness']]

    return df



---

Criação df com features

In [ ]:
features = create_df(audio_features)
features



---

Normalização dos dados

In [ ]:
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)
df_scaled



---

Função para testar quantos clusters utilizar utilizando WCSS (Gráfico cotovelo)

In [ ]:
numeric_features = df_scaled.select_dtypes(include=['number']) # Para não utilizar id da música

wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i,random_state=42)
    kmeans.fit(numeric_features)
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Número de Clusters')
plt.ylabel('WCSS')
plt.show()



---


Definição do número de clusters

In [ ]:
num_clusters = 4
print("Após analisar o gráfico verifique quantos clusters devem ser utilizados: ", num_clusters)



---

Função de criação de df com labels de clusters

In [ ]:
def cluster_dataframe(df, num_clusters, random_state=42):

  kmeans = KMeans(n_clusters=num_clusters, random_state=random_state)
  y_kmeans = kmeans.fit_predict(numeric_features)

  df['cluster'] = y_kmeans

  return df



---
Criação novo Dataframe com labels


In [ ]:
new_df = cluster_dataframe(df_scaled,int(num_clusters))
new_df.head()



---

Repartição Dataframes em dados (X) e labels (Y)

In [ ]:
X = new_df.drop(['cluster'], axis=1)
Y = new_df['cluster']


# Treinamento do modelo
---

Repartição do dataframe e treinamento dos modelos

In [ ]:
def train_and_evaluate_models(X, Y, n_splits=5, random_state=42):

  kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
  results = {
        'DecisionTree': {'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'auc': []},
        'RandomForest': {'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'auc': []},
        'XGBoost': {'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'auc': []},
    }

  for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

    trained_models  = {
        'DecisionTree': DecisionTreeClassifier(random_state=random_state),
        'RandomForest': RandomForestClassifier(random_state=random_state),
        'XGBoost': XGBClassifier(random_state=random_state)
    }

    for model_name, model in trained_models.items():
      model.fit(X_train, y_train)
      y_pred = model.predict(X_test)
      y_pred_proba = model.predict_proba(X_test)

      results[model_name]['accuracy'].append(accuracy_score(y_test, y_pred))
      results[model_name]['precision'].append(precision_score(y_test, y_pred, average='macro'))
      results[model_name]['recall'].append(recall_score(y_test, y_pred, average='macro'))
      results[model_name]['f1'].append(f1_score(y_test, y_pred, average='macro'))
      results[model_name]['auc'].append(roc_auc_score(y_test, y_pred_proba, multi_class='ovr'))

  return results, trained_models



---

Execução de treinamento

In [ ]:
results, trained_models=train_and_evaluate_models(X, Y, n_splits=5, random_state=42)



---

Plotagem de resultado aprendizado

In [ ]:
def plot_table_results(results):
  results_df = pd.DataFrame(results).T

  results_df['Acurácia Média'] = results_df['accuracy'].apply(lambda x: f"{np.mean(x)*100:.4f}%")
  results_df['Precisão Média'] = results_df['precision'].apply(lambda x: f"{np.mean(x)*100:.4f}%")
  results_df['Recall Médio'] = results_df['recall'].apply(lambda x: f"{np.mean(x)*100:.4f}%")
  results_df['F1-Score Médio'] = results_df['f1'].apply(lambda x: f"{np.mean(x)*100:.4f}%")
  results_df['AUC Média'] = results_df['auc'].apply(lambda x: f"{np.mean(x)*100:.4f}%")

  mean_metrics = results_df[['Acurácia Média', 'Precisão Média', 'Recall Médio', 'F1-Score Médio', 'AUC Média']]

  fig, ax = plt.subplots(figsize=(12, 3))
  ax.axis('off')
  ax.axis('tight')
  table = ax.table(cellText=mean_metrics.values, colLabels=mean_metrics.columns, rowLabels=mean_metrics.index, loc='center', cellLoc='center')
  table.auto_set_font_size(False)
  table.set_fontsize(12)

  for cell in table._cells:
    table._cells[cell].set_height(0.15)

  for key, cell in table.get_celld().items():
    cell.set_linewidth(1.5)

  plt.show()

plot_table_results(results)

# Seleção de modelo com maior rendimento



---

Cálculo de média ponderada, aplicando como acurácia ponto prioritário, em seguida precisão e recall. Por fim F1-Score e auc.


In [ ]:
def weighted_average(results):
  weights = {'accuracy': 0.3, 'precision': 0.2, 'recall': 0.2, 'f1': 0.15, 'auc': 0.15}
  weighted_averages = {}

  for model_name, metrics in results.items():
    weighted_averages[model_name] = 0
    for metric, values in metrics.items():
      weighted_averages[model_name] += np.mean(values) * weights[metric]

  return weighted_averages

weighted_avg_results = weighted_average(results)
print("Weighted Average Results:")
for model, score in weighted_avg_results.items():
    print(f"{model}: {score*100:.4f}%")



---

Plotagem de matriz de confusão de modelo escolhido

In [ ]:
def plot_confusion_matrix_xgboost(trained_models, X, Y):

  xgboost_model = trained_models['XGBoost']

  y_pred = xgboost_model.predict(X)

  cm = confusion_matrix(Y, y_pred)

  plt.figure(figsize=(8, 6))
  sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
  plt.title("Matriz de Confusão - XGBoost")
  plt.xlabel("Predição")
  plt.ylabel("Verdadeiro")
  plt.show()

plot_confusion_matrix_xgboost(trained_models, X, Y)

# Visualização modelo XGBoost

---



Ordem de importância de features apresentadas pelo Modelo

In [ ]:
plot_importance(trained_models['XGBoost'])
plt.show()

# Teste de aplicação de recomendação musical


---

Coleta de Ids e nomes das musicas

In [ ]:
def get_playlist_track_info(playlist_id):
    track_info = []
    playlist_url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'

    params = {
        'limit': 100
    }

    while True:
        response = requests.get(playlist_url, headers=headers, params=params)
        if response.status_code != 200:
            print(f"Erro ao buscar faixas da playlist: {response.status_code}")
            break

        playlist_data = response.json()
        tracks = playlist_data.get('items', [])

        for item in tracks:
            track = item['track']
            if track:
                track_info.append({
                    'id': track['id'],
                    'name': track['name']
                })

        next_url = playlist_data.get('next')
        if next_url:
            playlist_url = next_url
        else:
            break

    return pd.DataFrame(track_info)



---

Execução de coletas dos ids e nomes da músicas

In [ ]:
df_infos_musicas = get_playlist_track_info(playlist_id)
df_infos_musicas



---
Função para realizar a predição e recomendação de músicas com maiores similaridades

In [ ]:
def predict_cluster_and_find_similar(track_id, df, model, scaler):
    track_features = get_audio_features_for_track(track_id)
    if track_features is None:
        return None

    new_track_df = create_df([track_features])

    new_track_scaled = pd.DataFrame(scaler.transform(new_track_df), columns=new_track_df.columns)

    predicted_cluster = model.predict(new_track_scaled)[0]

    tracks_in_same_cluster = df[df['cluster'] == predicted_cluster]

    distances = euclidean_distances(new_track_scaled, tracks_in_same_cluster.drop('cluster', axis=1))

    tracks_in_same_cluster = df[df['cluster'] == predicted_cluster].copy()
    tracks_in_same_cluster['distance'] = distances[0]

    tracks_in_same_cluster_sorted = tracks_in_same_cluster.sort_values('distance')

    return tracks_in_same_cluster_sorted




---

Montagem de Dataframe que cruza index com nomes das músicas

In [ ]:
def get_similar_track_names(similar_tracks, df_infos_musicas):
  top_10_track_ids = similar_tracks.index[:10].to_list()
  similar_track_names = []

  for i, track_id in enumerate(top_10_track_ids):
      track_name = df_infos_musicas.loc[df_infos_musicas.index == track_id, 'name'].values
      if len(track_name) > 0:
          similar_track_names.append(track_name[0])
          print(f"{i+1}. {track_name[0]}")



---

Escolha a música inicial

In [ ]:
track_id_input = input("Insira o ID da música: ")



---
Coleta de lista de músicas similares modelo XGBoost

In [ ]:
similar_tracks_xgboost = predict_cluster_and_find_similar(track_id_input, new_df, trained_models['XGBoost'], scaler)

print("\nNomes das 10 músicas mais similares utilizando modelo XGBoost:")
get_similar_track_names(similar_tracks_xgboost,df_infos_musicas)